# **Patching Surface for *snappyHexMesh* Generation**
<hr>

In [1]:
from vmtk import vmtkscripts
import numpy as np
import os

The default current directory when loading a notebook from GDrive is the home folder of the current user. To load the user's libraries and modules, first change to the file location using the os' python module, and load the necessary modules, then change to the former directory. 

In [2]:
# Path where files are stored
aneurysmsPath = "/home/iagolessa/Documents/"

In [3]:
# Change to directory where vmtk_functions and vmtk_filenames are located
os.chdir(aneurysmsPath+"aneurysms/vmtkScriptsForAneurysms")

# Import the necessary user's modules
import vmtk_functions
from vmtk_filenames import *

# Change it back
os.chdir('/home/iagolessa')

<hr>

# **Surface Orientation and Patching**

The cells below takes care of orienting the surface by translate and rotate transformations, to conform the inlet patch surface normal to the opposite z direction (this orientation was chosen to facilitate the introduction of boundary conditions at simulation time). 

The oriented surface is then 'exploded' to its constituent patches using the 'vmtkthreshold' script. First, the boundary is inspected generating a reference system file -- 'referenceSystem.dat' with the normal and center information of each cap patch (the wall has id 1 by default, and we prefer to keep it that way). The capped surface is, then, transformed and saved to disk -- 'surfaceOriented.stl' -- and finally is decomposed into its patches.

One of the cells below try to identify the inlet patch index by the patch area, because, usually, the inlet has larger area than the outlets (but is always important to be sure of the inlet id).

In [61]:
# Path to save the generated files when extracting the surface
# We reccomend to separate it from the DICOM original dir. 
# Note that inside casePath, this code will create different sub-directories to store different data types
casePath       = aneurysmsPath+"aneurysms/geometries/einsteinCases/rupturedCases/vmtkReconstruction/case4/"

# Define subdirs
imagesDir       = casePath+'images/'
surfacesDir     = casePath+'surfaces/'
centerlinesDir  = casePath+'centerlines/'
meshesDir       = casePath+'meshes/'
parentVesselDir = casePath+"parentVessel/"

print('Files saved to:', casePath)

Files saved to: /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/rupturedCases/vmtkReconstruction/case4/


In [62]:
# Get array with pacthes info
# surface = vmtk_functions.readSurface(surfacesDir+'surfaceRemeshedSpatialVaryingResolution.stl')
surface = vmtk_functions.readSurface(surfacesDir+surfaceCappedFile)

Reading VTK XML surface file.


In [63]:
vmtk_functions.viewSurface(surface)

Quit renderer


In [64]:
# Capping the surface
surfaceCapper = vmtkscripts.vmtkSurfaceCapper()
surfaceCapper.Surface = surface
surfaceCapper.Method = 'centerpoint'
surfaceCapper.Interactive = 0
surfaceCapper.Execute()

In [65]:
# Surface boundary inspector
# Conveting surface to mesh to get cell entity ids
surfaceToMesh = vmtkscripts.vmtkSurfaceToMesh()
surfaceToMesh.Surface = surfaceCapper.Surface
surfaceToMesh.Execute()

# Inspecting
surfaceBoundaryInspector = vmtkscripts.vmtkMeshBoundaryInspector()
surfaceBoundaryInspector.Mesh = surfaceToMesh.Mesh
surfaceBoundaryInspector.CellEntityIdsArrayName = surfaceCapper.CellEntityIdsArrayName
surfaceBoundaryInspector.Execute()

# Store patch info in python dictionary using vmtksurfacetonumpy
vmtkToNumpy = vmtkscripts.vmtkSurfaceToNumpy()
vmtkToNumpy.Surface = surfaceBoundaryInspector.ReferenceSystems
vmtkToNumpy.Execute()

dictPatchData = vmtkToNumpy.ArrayDict


CellEntityId: 2
  Origin: 21.555721, -12.979963, 21.950214
  Normal: 0.186280, -0.921705, -0.340235
  Radius: 0.960764

CellEntityId: 3
  Origin: -0.000021, 0.000066, -2.336128
  Normal: 0.000000, -0.000000, -1.000000
  Radius: 2.359390

CellEntityId: 4
  Origin: -5.953386, -3.164332, 22.839617
  Normal: -0.834204, -0.479235, 0.272833
  Radius: 1.033402

CellEntityId: 5
  Origin: 29.735121, -0.088662, 18.659039
  Normal: 0.841543, 0.527252, -0.117522
  Radius: 1.123292

Quit renderer
wrapping vtkPolyData object
converting cell data: 
converting points
converting point data: 
BoundaryNormals
BoundaryRadius
Point1Array
Point2Array
CellEntityIds
converting cell connectivity list


In [66]:
# # Build condition array where centers are not equal to inlet center
# # therefore, outlet centers
# inletBarycenterArray = dictPatchData['Points'][int(inletPos)]
# notInlet = (dictPatchData['Points'] != inletBarycenterArray)

# # Inlet and outlet centers
# # inletBarycenters  = dictPatchData['Points'][int(index)].tolist()
# outletBarycenters = np.extract(notInlet, dictPatchData['Points']).tolist()

In [67]:
# Get inlet by maximum radius condition
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Get max radius and its index
maxRadius = max(dictPatchData['PointData']['BoundaryRadius'])
inletPos, = np.where( dictPatchData['PointData']['BoundaryRadius'] == maxRadius )

# Number of patches
nPatches  = len(dictPatchData['PointData']['CellEntityIds'])

# Update dictPatchData with patch info
dictPatchData['PatchType'] = np.array(nPatches*['outlet'])
dictPatchData['PatchType'][int(inletPos)] = 'inlet'
dictPatchData

{'Points': array([[  2.15557213e+01,  -1.29799633e+01,   2.19502144e+01],
        [ -2.11906572e-05,   6.55368131e-05,  -2.33612776e+00],
        [ -5.95338631e+00,  -3.16433167e+00,   2.28396168e+01],
        [  2.97351208e+01,  -8.86624232e-02,   1.86590385e+01]], dtype=float32),
 'PointData': {'BoundaryNormals': array([[  1.86280392e-01,  -9.21704868e-01,  -3.40234849e-01],
         [  0.00000000e+00,  -1.41034145e-33,  -1.00000000e+00],
         [ -8.34203707e-01,  -4.79235178e-01,   2.72832952e-01],
         [  8.41542864e-01,   5.27251539e-01,  -1.17522007e-01]]),
  'BoundaryRadius': array([ 0.96076446,  2.35939015,  1.03340235,  1.12329209]),
  'Point1Array': array([[  2.15557214e+01,  -1.33126727e+01,   2.28515324e+01],
         [ -2.11906574e-05,  -2.35932461e+00,  -2.33612776e+00],
         [ -5.95338651e+00,  -2.65305610e+00,   2.37376798e+01],
         [  2.97351203e+01,  -3.33042098e-01,   1.75626522e+01]]),
  'Point2Array': array([[  2.06117736e+01,  -1.31478614e+01,   2.

In [68]:
wallId  = 1
inletId = dictPatchData['PointData']['CellEntityIds'][int(inletPos)]

lastId = np.max(dictPatchData['PointData']['CellEntityIds'])

wallId, inletId, lastId

(1, 3, 5)

In [69]:
# Counting the number of inlets and outlets
# using the numpy.unique function: it returns an ndarray with the unique itens of the input array
# with the return_counts=True also returns the number of appearance of each item
patchTypes, numberOfPatches = np.unique(dictPatchData['PatchType'], return_counts=True)

# Zip both patchTypes and numberOfPatches and convert to dictionary
# with number of inlets and outlets
patchCount = dict(zip(patchTypes, numberOfPatches))
patchCount

{'inlet': 1, 'outlet': 3}

In [70]:
# Surface reorientation
# It is better to reorient the final capped surface before sending to mesh in SnappyHexMesh
# The vmtksurfacereferencesystemtransform script takes a surface (the surfaceCapped above) 
# and rotates and translates it conforming one of its patches (in our case the inlet patch) to a target reference system
# for that, it uses the output of the vtmkboundaryinspector

surfaceTransform = vmtkscripts.vmtkSurfaceReferenceSystemTransform()

# or use the just created surfaceCapped
surfaceTransform.Surface = surfaceToMesh.Surface

# Target reference system parameters
surfaceTransform.ReferenceOrigin  = [0, 0, 0]  # translate to origin of system
surfaceTransform.ReferenceNormal1 = [0, 0, -1] # inlet normal will coincide with -z axis orientation
surfaceTransform.ReferenceNormal2 = [0, 0, -1]

# Surface reference system
surfaceTransform.ReferenceSystems = surfaceBoundaryInspector.ReferenceSystems

# to get the reference systems of inlet patch
# Note that, if there is more than one inlet, the inlet chose is the one with largest area
surfaceTransform.ReferenceSystemId = inletId

surfaceTransform.ReferenceSystemsIdArrayName      = surfaceCapper.CellEntityIdsArrayName
surfaceTransform.ReferenceSystemsNormal1ArrayName = 'BoundaryNormals'
surfaceTransform.ReferenceSystemsNormal2ArrayName = 'BoundaryNormals'

surfaceTransform.SurfaceOutputFileName = surfacesDir+surfaceOrientedFile
surfaceTransform.Execute()
surfaceTransform.IOWrite()

# Debug
# surfaceTransform.PrintInputMembers()
# surfaceTransform.PrintOutputMembers()

Writing VTK XML surface file.


In [71]:
# Check if patch dir exists
# If it not exists, create it
# the patch dir will contains the surface patch files for meshing (wall, inlet and outlet(s)) 

patchDir = surfacesDir+'patches/'

if not os.path.isdir(patchDir):
    os.makedirs(patchDir)
    print(patchDir+' created.')

In [72]:
# Using vmtkThreshold script to extract patches for mesh generations in snappy

# Surface name to decompose if needs to be read before
# if you uncomment these next two lines, dont forget to change line 22 below
# surfaceOriented = readSurface(surfacesDir+surfaceCappedFile)

# File names for patches
wallFileName = patchDir+'wall.stl'

# Extracting first the wall
extractThreshold = vmtkscripts.vmtkThreshold()

extractThreshold.Surface = surfaceTransform.Surface
extractThreshold.LowThreshold  = wallId
extractThreshold.HighThreshold = wallId
extractThreshold.SurfaceOutputFileName = wallFileName

print('Extracting surface with id ', wallId)

extractThreshold.Execute()
extractThreshold.IOWrite()
extractThreshold.OutputText('Patch saved in '+extractThreshold.SurfaceOutputFileName+'\n')

# Outlet initial index (to increment) and to be used in output filename
outletIndex = 1
inletIndex  = 1

# Loop to extract inlet and outlet patches
for i in np.arange(nPatches):
    
    print('Extracting surface with id ', dictPatchData['PointData']['CellEntityIds'][i])
    
    # Instantiate vmtkthreshold
    extractThreshold = vmtkscripts.vmtkThreshold()
    extractThreshold.Surface       = surfaceTransform.Surface
    extractThreshold.LowThreshold  = dictPatchData['PointData']['CellEntityIds'][i]
    extractThreshold.HighThreshold = dictPatchData['PointData']['CellEntityIds'][i]
    
    # Defining output file names
    if dictPatchData['PatchType'][i] == 'inlet':
        
        if patchCount['inlet'] == 1:
            inletFileName = patchDir+'inlet.stl'
        else:
            inletFileName = patchDir+'inlet'+str(inletIndex)+'.stl'
            inletIndex += 1
            
        # Attribute surface output file name
        print('Patch with id '+str(dictPatchData['PointData']['CellEntityIds'][i])+' is an inlet. Defining file name: '+inletFileName) # debugging
        
        extractThreshold.SurfaceOutputFileName = inletFileName
        
    elif dictPatchData['PatchType'][i] == 'outlet':
        
        if patchCount['outlet'] == 1:
            outletFileName = patchDir+'outlet.stl'
            
        else:
            outletFileName = patchDir+'outlet'+str(outletIndex)+'.stl'
            outletIndex += 1
            
        print('Patch with id '+str(dictPatchData['PointData']['CellEntityIds'][i])+' is an outlet. Defining file name: '+outletFileName) # debugging
        
        extractThreshold.SurfaceOutputFileName = outletFileName
    
    extractThreshold.Execute()
    extractThreshold.OutputText('Patch saved in '+extractThreshold.SurfaceOutputFileName+'\n')
    extractThreshold.IOWrite()
    print('\n')

Extracting surface with id  1
Writing STL surface file.
Patch saved in /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/rupturedCases/vmtkReconstruction/case4/surfaces/patches/wall.stl
Extracting surface with id  2
Patch with id 2 is an outlet. Defining file name: /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/rupturedCases/vmtkReconstruction/case4/surfaces/patches/outlet1.stl
Patch saved in /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/rupturedCases/vmtkReconstruction/case4/surfaces/patches/outlet1.stl
Writing STL surface file.


Extracting surface with id  3
Patch with id 3 is an inlet. Defining file name: /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/rupturedCases/vmtkReconstruction/case4/surfaces/patches/inlet.stl
Patch saved in /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/rupturedCases/vmtkReconstruction/case4/surfaces/patches/inlet.stl
Writing STL surface file.


Extracting surface with id  4
Patch with id 4 

<hr>
The two cells below save the oriented surface and its Voronoi diagram to the parent vessel reconstruction procedure 

In [ ]:
# Final surface for parent vessel reconstruction
caseId = 'case11'

if not os.path.isdir(parentVesselDir+caseId):
    os.makedirs(parentVesselDir+caseId)
    
surfaceForParentVessel = parentVesselDir+caseId+"/"+caseId+"_model.vtp"
print(surfaceForParentVessel)
vmtk_functions.writeSurface(surfaceTransform.Surface, surfaceForParentVessel, "binary")

In [ ]:
# Compute Voronoi Diagram Separately for parent vessel reconstruction
voronoiDiagram = vmtkscripts.vmtkDelaunayVoronoi()

voronoiDiagram.Surface = surfaceTransform.Surface
# voronoiDiagram.SurfaceInputFileName = surfaceInputFile #casePath+"surfaces/surfaceRemeshed.vtp" 
# voronoiDiagram.IORead()
voronoiDiagram.CheckNonManifold = 1
voronoiDiagram.RemoveSubresolutionTetrahedra = 1
# voronoiDiagram.DelaunayTessellationOutputFileName = casePath+"surfaces/delaunayTesselation.vtp"
voronoiDiagram.VoronoiDiagramOutputFileName = parentVesselDir+caseId+"/"+caseId+"_voronoi.vtp"
voronoiDiagram.Execute()
voronoiDiagram.IOWrite()

<hr>

The following cell puts together all the operations above in a single function.
I am considering changing it to a class and define it as VMTK script or a simple python script.

Because of the VMTK piping system when used in my workflow of aneurysm surface extraction, 
it would be nice I think to include it as a specialized pypes script.

In [54]:
def patchSurface(surface, outputDir):
    """ 
    Function to patch a surface to be used 
    in CFD by creating a mesh in snappyHexMesh.
    The function also orient the surface putting
    the inlet (selected by largest radius) center
    on the origin of the system, with its outward
    normal point to -z direction.
    """
    # Cap surface
    surfaceCapper = vmtkscripts.vmtkSurfaceCapper()
    surfaceCapper.Surface = surface
    surfaceCapper.Method = 'centerpoint'
    surfaceCapper.Interactive = 0
    surfaceCapper.Execute()

    # Surface boundary inspector
    # Conveting surface to mesh to get cell entity ids
    surfaceToMesh = vmtkscripts.vmtkSurfaceToMesh()
    surfaceToMesh.Surface = surfaceCapper.Surface
    surfaceToMesh.Execute()

    # Inspecting
    surfaceBoundaryInspector = vmtkscripts.vmtkMeshBoundaryInspector()
    surfaceBoundaryInspector.Mesh = surfaceToMesh.Mesh
    surfaceBoundaryInspector.CellEntityIdsArrayName = surfaceCapper.CellEntityIdsArrayName
    surfaceBoundaryInspector.Execute()

    # Store patch info in python dictionary using vmtksurfacetonumpy
    vmtkToNumpy = vmtkscripts.vmtkSurfaceToNumpy()
    vmtkToNumpy.Surface = surfaceBoundaryInspector.ReferenceSystems
    vmtkToNumpy.Execute()

    dictPatchData = vmtkToNumpy.ArrayDict

    # Get inlet by maximum radius condition
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Get max radius and its index
    maxRadius = max(dictPatchData['PointData']['BoundaryRadius'])
    inletPos, = np.where( dictPatchData['PointData']['BoundaryRadius'] == maxRadius )

    # Number of patches
    nPatches  = len(dictPatchData['PointData']['CellEntityIds'])

    # Update dictPatchData with patch info
    dictPatchData['PatchType'] = np.array(nPatches*['outlet'])
    dictPatchData['PatchType'][int(inletPos)] = 'inlet'
    dictPatchData

    wallId  = 1
    inletId = dictPatchData['PointData']['CellEntityIds'][int(inletPos)]

    # Counting the number of inlets and outlets
    # using the numpy.unique function: it returns an ndarray with the unique itens of the input array
    # with the return_counts=True also returns the number of appearance of each item
    patchTypes, numberOfPatches = np.unique(dictPatchData['PatchType'], return_counts=True)

    # Zip both patchTypes and numberOfPatches and convert to dictionary
    # with number of inlets and outlets
    patchCount = dict(zip(patchTypes, numberOfPatches))

    # Surface reorientation
    # It is better to reorient the final capped surface before sending to mesh in SnappyHexMesh
    # The vmtksurfacereferencesystemtransform script takes a surface (the surfaceCapped above) 
    # and rotates and translates it conforming one of its patches (in our case the inlet patch) to a target reference system
    # for that, it uses the output of the vtmkboundaryinspector

    surfaceTransform = vmtkscripts.vmtkSurfaceReferenceSystemTransform()

    # or use the just created surfaceCapped
    surfaceTransform.Surface = surfaceToMesh.Surface

    # Target reference system parameters
    surfaceTransform.ReferenceOrigin  = [0, 0, 0]  # translate to origin of system
    surfaceTransform.ReferenceNormal1 = [0, 0, -1] # inlet normal will coincide with -z axis orientation
    surfaceTransform.ReferenceNormal2 = [0, 0, -1]

    # Surface reference system
    surfaceTransform.ReferenceSystems = surfaceBoundaryInspector.ReferenceSystems

    # to get the reference systems of inlet patch
    # Note that, if there is more than one inlet, the inlet chose is the one with largest area
    surfaceTransform.ReferenceSystemId = inletId

    surfaceTransform.ReferenceSystemsIdArrayName      = surfaceCapper.CellEntityIdsArrayName
    surfaceTransform.ReferenceSystemsNormal1ArrayName = 'BoundaryNormals'
    surfaceTransform.ReferenceSystemsNormal2ArrayName = 'BoundaryNormals'

#     surfaceTransform.SurfaceOutputFileName = surfacesDir+surfaceOrientedFile
    surfaceTransform.Execute()
#     surfaceTransform.IOWrite()

    # Check if patch dir exists
    # If it not exists, create it
    # the patch dir will contains the surface patch files for meshing (wall, inlet and outlet(s)) 

    patchDir = outputDir+'patches/'

    if not os.path.isdir(patchDir):
        os.makedirs(patchDir)
        print(patchDir+' created.')

    # Using vmtkThreshold script to extract patches for mesh generations in snappy

    # Surface name to decompose if needs to be read before
    # if you uncomment these next two lines, dont forget to change line 22 below
    # surfaceOriented = readSurface(surfacesDir+surfaceCappedFile)

    # File names for patches
    wallFileName = patchDir+'wall.stl'

    # Extracting first the wall
    extractThreshold = vmtkscripts.vmtkThreshold()

    extractThreshold.Surface = surfaceTransform.Surface
    extractThreshold.LowThreshold  = wallId
    extractThreshold.HighThreshold = wallId
    extractThreshold.SurfaceOutputFileName = wallFileName

    print('Extracting surface with id ', wallId)

    extractThreshold.Execute()
    extractThreshold.IOWrite()
    extractThreshold.OutputText('Patch saved in '+extractThreshold.SurfaceOutputFileName+'\n')

    # Outlet initial index (to increment) and to be used in output filename
    outletIndex = 1
    inletIndex  = 1

    # Loop to extract inlet and outlet patches
    for i in np.arange(nPatches):

        print('Extracting surface with id ', dictPatchData['PointData']['CellEntityIds'][i])

        # Instantiate vmtkthreshold
        extractThreshold = vmtkscripts.vmtkThreshold()
        extractThreshold.Surface       = surfaceTransform.Surface
        extractThreshold.LowThreshold  = dictPatchData['PointData']['CellEntityIds'][i]
        extractThreshold.HighThreshold = dictPatchData['PointData']['CellEntityIds'][i]

        # Defining output file names
        if dictPatchData['PatchType'][i] == 'inlet':

            if patchCount['inlet'] == 1:
                inletFileName = patchDir+'inlet.stl'
            else:
                inletFileName = patchDir+'inlet'+str(inletIndex)+'.stl'
                inletIndex += 1

            # Attribute surface output file name
            print('Patch with id '+str(dictPatchData['PointData']['CellEntityIds'][i])+' is an inlet. Defining file name: '+inletFileName) # debugging

            extractThreshold.SurfaceOutputFileName = inletFileName

        elif dictPatchData['PatchType'][i] == 'outlet':

            if patchCount['outlet'] == 1:
                outletFileName = patchDir+'outlet.stl'

            else:
                outletFileName = patchDir+'outlet'+str(outletIndex)+'.stl'
                outletIndex += 1

            print('Patch with id '+str(dictPatchData['PointData']['CellEntityIds'][i])+' is an outlet. Defining file name: '+outletFileName) # debugging

            extractThreshold.SurfaceOutputFileName = outletFileName

        extractThreshold.Execute()
        extractThreshold.OutputText('Patch saved in '+extractThreshold.SurfaceOutputFileName+'\n')
        extractThreshold.IOWrite()
        print('\n')